In [3]:
# Installing Nvidia Apex
#!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ../input/nvidiaapex/repository/NVIDIA-apex-39e153a

#pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" NVIDIA-apex-39e153a


In [4]:
import sys
import os
import gc
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
#from nltk.stem import PorterStemmer
from sklearn.metrics import roc_auc_score
from sklearn import metrics
%load_ext autoreload
%autoreload 2

from tqdm import tqdm, tqdm_notebook
import warnings
warnings.filterwarnings(action='once')
import pickle

from apex import amp
import shutil
import regex as re

device=torch.device('cuda')

#package_dir_a = '../input/pytorchpretrainedgpt2/pytorch-pretrained-gpt2/pytorch-pretrained-gpt2/'
#package_dir_a = '../data/pytorch-pretrained-gpt2/'
#package_dir_a = '../data/pytorch-pretrained-BERT_latest/'

package_dir_a = '../data/pytorch-pretrained-gpt2/'
sys.path.insert(0, package_dir_a)

from pytorch_pretrained_bert import GPT2Tokenizer, GPT2ClassificationHeadModel
from pytorch_pretrained_bert import OpenAIAdam


# This is the GPT2 configuration file
from pytorch_pretrained_bert import GPT2Config

from sklearn.model_selection import StratifiedKFold

/opt/anaconda3/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, MutableMapping
/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [5]:
Data_dir = '../data/'
Input_dir = Data_dir
WORK_DIR = '.'

FOLD_NUM = 0
#Data_dir="../input/jigsaw-unintended-bias-in-toxicity-classification/"
#Input_dir = Data_dir
#WORK_DIR = '.'

SEED = 1234
TRAIN_SAMPLE_SEED = 1234
MAX_SEQ_LENGTH = 220

#GPT2_MODEL_PATH = '../input/gpt2-models/'
GPT2_MODEL_PATH = '../data/gpt2-models_kaggle/'
    
TOKENIZER_PATH = '../data/gpt2-models_kaggle/'
GPT2_CONFIG_PATH = '../data/gpt2-models_kaggle/config.json'

shutil.copyfile(GPT2_MODEL_PATH + 'config.json', WORK_DIR + 'config.json')

num_to_load= 1804874 #1000000 #1704874 
#valid_size=  100000 
TOXICITY_COLUMN = 'target'

#gpt2_config = GPT2Config('../input/gpt2-models/config.json')
gpt2_config = GPT2Config(GPT2_CONFIG_PATH)


In [6]:
#train_df = pd.read_csv(os.path.join(Data_dir,"train.csv")).sample(num_to_load+valid_size,random_state=TRAIN_SAMPLE_SEED)
train_df = pd.read_csv(os.path.join(Data_dir,"train.csv"))
print('Train: loaded %d records' % len(train_df))

# Make sure all comment_text values are strings
train_df['comment_text'] = train_df['comment_text'].astype(str) 

Train: loaded 1804874 records


In [7]:
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(tokens_a)+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

#tokenizer = GPT2Tokenizer.from_pretrained('../input/gpt2-models/')

tokenizer = GPT2Tokenizer.from_pretrained(TOKENIZER_PATH)
sequences = convert_lines(train_df['comment_text'], MAX_SEQ_LENGTH, tokenizer)

../data/pytorch-pretrained-gpt2/pytorch_pretrained_bert/tokenization_gpt2.py:146: ResourceWarning: unclosed file <_io.TextIOWrapper name='../data/gpt2-models_kaggle/vocab.json' mode='r' encoding='UTF-8'>
  self.encoder = json.load(open(vocab_file))
../data/pytorch-pretrained-gpt2/pytorch_pretrained_bert/tokenization_gpt2.py:151: ResourceWarning: unclosed file <_io.TextIOWrapper name='../data/gpt2-models_kaggle/merges.txt' mode='r' encoding='utf-8'>
  bpe_data = open(merges_file, encoding='utf-8').read().split('\n')[1:-1]



47658


In [8]:
train_df=train_df.fillna(0)
# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
y_columns=['target']

train_df = train_df.drop(['comment_text'],axis=1)
# convert target to 0,1
train_df['target']=(train_df['target']>=0.5).astype(float)  


TOXICITY_COLUMN = 'target'
identity_columns = ['asian', 'atheist', 
       'bisexual', 'black', 'buddhist', 'christian', 'female',
       'heterosexual', 'hindu', 'homosexual_gay_or_lesbian',
       'intellectual_or_learning_disability', 'jewish', 'latino', 'male',
       'muslim', 'other_disability', 'other_gender',
       'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white']
    
subgroup_bool_train = train_df[identity_columns].fillna(0)>=0.5
toxic_bool_train = train_df[TOXICITY_COLUMN].fillna(0)>=0.5
subgroup_negative_mask = subgroup_bool_train.values.sum(axis=1).astype(bool) & ~toxic_bool_train

#bpsn = ((subgroup_bool_train.values.sum(axis=1).astype(bool) & ~toxic_bool_train)).astype(int)   +  \
# ((~subgroup_bool_train.values.sum(axis=1).astype(bool)  & toxic_bool_train)).astype(int) 


# Overall
weights = np.ones((len(train_df),))
# Subgroup negative
weights += subgroup_negative_mask #bpsn
loss_weight = 1.0 / weights.mean()

train_df['weights'] = weights

In [9]:
splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=47).split(sequences, train_df[y_columns].values[:,0]))

y_columns=['target', 'weights']
y_columns = ['target', 'weights', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', 'sexual_explicit']

len_train = 0
len_val = 0
for fld, (train_index, val_index) in enumerate(splits):
    if fld == FOLD_NUM:
        len_train = len(train_index)
        len_val = len(val_index)
        break
        
train_idx = np.zeros((len_train))
val_idx = np.zeros((len_val))

for fld, (train_index, val_index) in enumerate(splits):
    if fld == FOLD_NUM:
        train_idx = train_index
        val_idx = val_index
        break
        
X = sequences[train_idx]
y = train_df[y_columns].values[train_idx]

X_val = sequences[val_index]
y_val = train_df[y_columns].values[val_index]

print(X.shape, y.shape, X_val.shape, y_val.shape, train_df.shape)

(1443899, 218) (1443899, 8) (360975, 218) (360975, 8) (1804874, 45)


In [10]:
# y_columns=['target', 'weights']
# y_columns = ['target', 'weights', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', 'sexual_explicit']
    
# X = sequences[:num_to_load]                
# y = train_df[y_columns].values[:num_to_load]
# X_val = sequences[num_to_load:]                
# y_val = train_df[y_columns].values[num_to_load:]

# print(X.shape, y.shape, X_val.shape, y_val.shape, train_df.shape)

In [11]:
# test_df=train_df.tail(valid_size).copy()
# train_df=train_df.head(num_to_load)
# print(X.shape, y.shape, X_val.shape, y_val.shape, train_df.shape, test_df.shape)

In [12]:
from torch.utils import data
from tqdm import tqdm_notebook as tqdm

class LenMatchBatchSampler(data.BatchSampler):
    def __iter__(self):

        buckets = [[]] * 100
        yielded = 0

        for idx in self.sampler:
            count_zeros = torch.sum(self.sampler.data_source[idx][0] == 0)
            count_zeros = int(count_zeros / 64) 
            if len(buckets[count_zeros]) == 0:  buckets[count_zeros] = []

            buckets[count_zeros].append(idx)

            if len(buckets[count_zeros]) == self.batch_size:
                batch = list(buckets[count_zeros])
                yield batch
                yielded += 1
                buckets[count_zeros] = []

        batch = []
        leftover = [idx for bucket in buckets for idx in bucket]

        for idx in leftover:
            batch.append(idx)
            if len(batch) == self.batch_size:
                yielded += 1
                yield batch
                batch = []

        if len(batch) > 0 and not self.drop_last:
            yielded += 1
            yield batch

        assert len(self) == yielded, "produced an incorrect number of batches. expected %i, but yielded %i" %(len(self), yielded)

def trim_tensors(tsrs):
    max_len = torch.max(torch.sum( (tsrs[0] != 0  ), 1))
    if max_len > 2: 
        tsrs = [tsr[:, :max_len] for tsr in tsrs] 
    return tsrs 

In [13]:
output_model_file = "gpt2_pytorch_mymodel_fold"
output_model_file = output_model_file + str(FOLD_NUM) + '.bin'


lr= 2e-5 # 2e-5
batch_size = 32
EPOCHS = 1
accumulation_steps=1
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


model = GPT2ClassificationHeadModel.from_pretrained(GPT2_MODEL_PATH, clf_dropout=0.2, n_class=len(y_columns))

model.zero_grad()
model = model.to(device)
param_optimizer = list(model.named_parameters())

In [14]:

train_dataset = torch.utils.data.TensorDataset(torch.tensor(X,dtype=torch.long),
                                            torch.from_numpy(np.array(X>0, dtype=np.uint8)),
                                            torch.zeros(X.shape),
                                            torch.tensor(y,dtype=torch.float))

ran_sampler = torch.utils.data.RandomSampler(train_dataset)

len_sampler = LenMatchBatchSampler(ran_sampler, batch_size = batch_size, drop_last = False)


In [15]:
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]


num_train_optimization_steps = int(EPOCHS*len(train_dataset)/batch_size/accumulation_steps)

optimizer = OpenAIAdam(optimizer_grouped_parameters, 
                       lr=lr,
                       warmup=0.05,
                       t_total=num_train_optimization_steps)

model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)

In [16]:
loss_weight

0.9269002138952298

In [17]:
gc.collect()

348

In [18]:
torch.cuda.empty_cache()

In [19]:
def custom_loss(data, targets):  #data=predictions, target = groundtruth
    ''' Define custom loss function for weighted BCE on 'target' column '''
    bce_loss_1 = nn.BCEWithLogitsLoss(weight=targets[:,1:2])(data[:,:1],targets[:,:1]) #target
    bce_loss_2 = nn.BCEWithLogitsLoss()(data[:,2:],targets[:,2:]) #aux targets
    #bce_loss_2  = nn.KLDivLoss()(torch.log_softmax(data[:,2:],1),targets[:,2:]) #aux targets
    return (bce_loss_1 * loss_weight) + bce_loss_2
    
    
    
#UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
#warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."

In [20]:
def mygc():
    gc.collect()
    return None

tq = tqdm_notebook(range(EPOCHS))
for epoch in tq:
    #train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler = len_sampler) 
    
    avg_loss = 0.
    avg_accuracy = 0.
    lossf=None
    tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
    #for i,(x_batch, y_batch) in tk0:
    for i,batch in tk0: 
        tsrs = trim_tensors(batch)
        b_input_ids, b_input_mask, b_segment_ids, b_label = tuple(t.to(device) for t in tsrs)
        
        
        #optimizer.zero_grad()
        #y_pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
        #loss = custom_loss(y_pred,y_batch.to(device))
        
        y_pred = model(b_input_ids.to(device))
        loss = custom_loss(y_pred,b_label.to(device))
        
        #del y_pred,y_batch
        gc.collect()
        #break
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        
         
        
        if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
            optimizer.step()                            # Now we can do an optimizer step
            optimizer.zero_grad()
        if lossf:
            lossf = 0.98*lossf+0.02*loss.item()
        else:
            lossf = loss.item()
        tk0.set_postfix(loss = lossf)
        avg_loss += loss.item() / len(train_loader)
        #avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (y_batch[:,0]>0.5).to(device)).to(torch.float) ).item()/len(train_loader)
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (b_label[:,0]>0.5).to(device)).to(torch.float) ).item()/len(train_loader)
        
        mygc()
        torch.cuda.empty_cache()
        
        #if (i>300):
        #    break
    
    tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)
 
    
torch.save(model.state_dict(), output_model_file) 
#100% 1/1 [4:32:01<00:00, 16321.62s/it, avg_accuracy=0.945, avg_loss=0.225]


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [21]:
gc.collect()
torch.cuda.empty_cache() 
gc.collect() 

0

In [22]:
# Run validation
# The following 2 lines are not needed but show how to download the model for prediction
model = GPT2ClassificationHeadModel.from_pretrained(GPT2_MODEL_PATH, clf_dropout=0.2, n_class=len(y_columns))
model.load_state_dict(torch.load(output_model_file ))
model.to(device)
for param in model.parameters():
    param.requires_grad=False
model.eval()

GPT2ClassificationHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0): Block(
        (ln_1): FusedLayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
        )
        (ln_2): FusedLayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
        )
      )
      (1): Block(
        (ln_1): FusedLayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
        )
        (ln_2): FusedLayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
        )
      )
      (2): Block(
        (ln_1): FusedLayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=T

In [23]:
valid_preds = np.zeros((len(X_val)))
valid = torch.utils.data.TensorDataset(torch.tensor(X_val,dtype=torch.long))
valid_loader = torch.utils.data.DataLoader(valid, batch_size=32, shuffle=False)

tk0 = tqdm_notebook(valid_loader)
for i,(x_batch,)  in enumerate(tk0):
    pred = model(x_batch.to(device))
    valid_preds[i*32:(i+1)*32]=pred[:,0].detach().cpu().squeeze().numpy()

In [24]:
# From baseline kernel

def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]>0.5
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)



SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]>0.5]
    return compute_auc((subgroup_examples[label]>0.5), subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[(df[subgroup]>0.5) & (df[label]<=0.5)]
    non_subgroup_positive_examples = df[(df[subgroup]<=0.5) & (df[label]>0.5)]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[(df[subgroup]>0.5) & (df[label]>0.5)]
    non_subgroup_negative_examples = df[(df[subgroup]<=0.5) & (df[label]<=0.5)]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]>0.5])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)

In [25]:
MODEL_NAME = 'model1'
test_df = train_df.iloc[val_idx].copy()

identity_columns = [
     'male',  'female',  'homosexual_gay_or_lesbian', 'christian', 'jewish','muslim', 'black', 'white', 'psychiatric_or_mental_illness' ]
   
test_df[MODEL_NAME]=torch.sigmoid(torch.tensor(valid_preds)).numpy()
TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, 'target')
print(bias_metrics_df)
get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, MODEL_NAME))




   bnsp_auc  bpsn_auc                       subgroup  subgroup_auc  \
2  0.960662  0.890127      homosexual_gay_or_lesbian      0.846586   
6  0.968825  0.873024                          black      0.854669   
7  0.964061  0.893505                          white      0.859274   
5  0.959257  0.914816                         muslim      0.875753   
4  0.950636  0.935091                         jewish      0.889653   
0  0.963746  0.942730                           male      0.928077   
3  0.946463  0.962159                      christian      0.929321   
1  0.960651  0.950449                         female      0.932405   
8  0.969740  0.935161  psychiatric_or_mental_illness      0.933825   

   subgroup_size  
2           2064  
6           2815  
7           4780  
5           3882  
4           1448  
0           8081  
3           7068  
1          10014  
8            785  


0.9348877675010141

In [27]:
results_oof = torch.sigmoid(torch.tensor(valid_preds)).numpy()


np.save('results_oof_fold' + str(FOLD_NUM) + '.npy',results_oof)

In [28]:
print(valid_preds.shape, results_oof.shape)

(360975,) (360975,)


In [29]:
#Inference on test set 
gc.collect()
torch.cuda.empty_cache()

test_df = pd.read_csv(os.path.join(Data_dir,"test.csv"))
test_df['comment_text'] = test_df['comment_text'].astype(str) 
X_test = convert_lines(test_df["comment_text"].fillna("DUMMY_VALUE"), MAX_SEQ_LENGTH, tokenizer)




2560


In [30]:
test_preds = np.zeros((len(X_test)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)
tk0 = tqdm(test_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device))
    test_preds[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

In [33]:
submission = pd.DataFrame.from_dict({
    'id': test_df['id'],
    'prediction': test_pred
})


submission.to_csv( 'submission_fold' + str(FOLD_NUM) + '.csv', index=False)

In [34]:
submission

,id,prediction
0,7000000,0.002611
1,7000001,0.000039
2,7000002,0.003795
3,7000003,0.000729
4,7000004,0.980431
5,7000005,0.000064
6,7000006,0.001694
7,7000007,0.002621
8,7000008,0.058993
9,7000009,0.020489
